In [1]:
from pypalace import Config, Model, Domains, Boundaries, Solver, Simulation

#### Look at the mesh attributes to help us define our config file below

In [2]:
meshfile = "tunable_xmon.bdf"
Simulation.get_mesh_attributes(meshfile)

,Name,ID,Type
0,substrate,1,Volume
1,air,2,Volume
2,xmon_cross,3,Surface
3,squid_loop,4,Surface
4,flux_line,5,Surface
5,flux_port,6,Surface
6,ground_plane,7,Surface
7,far_field,8,Surface


## Build a Palace config file for a magnetostatic simulation

#### Create config object (creates config["Problem"]) and add config["Model"]

In [3]:
my_config = Config("Magnetostatic",Output="magneto_output")
my_config.add_Model(meshfile,L0=1e-6) # no adaptive mesh refinement, already finely meshed 

#### define materials and boundary conditions

In [4]:
# define materials
silicon = Domains.Material([1],1.0,11.45,0.0) # silicon 
air = Domains.Material([2],1.0,1.0,0.0) # air
my_materials = [silicon,air] # material list for input into add_Domains()

# define boundary conditions
PECs = Boundaries.PEC([3,5,7,8]) # xmon cross, flux bias line, ground_plane, far_field
flux_port = Boundaries.SurfaceCurrent(Index=1,Attributes=[6],Direction="+X") # add surface current to the flux port to give flux line current 
my_BCs = [PECs,flux_port] # boundary condition list for input into add_Boundaries()

#### add surface flux postprocessing

In [5]:
# add our "dummy" SQUID loop to SurfaceFlux postprocesssing so we can get magnetic flux through it
surfaceFlux_pp = Boundaries.Postprocessing_SurfaceFlux(Index=2,Attributes=[4],Type="Magnetic")
my_Boundaries_pp = [surfaceFlux_pp]

#### add Domains (materials) and Boundaries (BC and BC postprocessing) to config["Domains"] and config["Boundaries"], respectively. 

In [6]:
# add config["Domains"] and config["Boundaries"] using our material and BC lists above
my_config.add_Domains(my_materials)
my_config.add_Boundaries(my_BCs,Postprocessing=my_Boundaries_pp)

#### add magnetostatic and linear solver parameters

In [7]:
magneto_params = Solver.Magnetostatic(Save=5)

Linear_params = Solver.Linear(Type="Default",
                              KSPType = "Default",
                              Tol = 1e-6,
                              MaxIts = 100)
                              
my_config.add_Solver(Simulation=magneto_params,Order= 2,Linear=Linear_params)

In [8]:
my_config.print_config()

{
  "Problem": {
    "Type": "Magnetostatic",
    "Verbose": 2,
    "Output": "magneto_output"
  },
  "Model": {
    "Mesh": "tunable_xmon.bdf",
    "L0": 1e-06
  },
  "Domains": {
    "Materials": [
      {
        "Attributes": [
          1
        ],
        "Permeability": 1.0,
        "Permittivity": 11.45,
        "LossTan": 0.0
      },
      {
        "Attributes": [
          2
        ],
        "Permeability": 1.0,
        "Permittivity": 1.0,
        "LossTan": 0.0
      }
    ]
  },
  "Boundaries": {
    "PEC": {
      "Attributes": [
        3,
        5,
        7,
        8
      ]
    },
    "SurfaceCurrent": [
      {
        "Index": 1,
        "Attributes": [
          6
        ],
        "Direction": "+X"
      }
    ],
    "Postprocessing": {
      "SurfaceFlux": [
        {
          "Index": 2,
          "Attributes": [
            4
          ],
          "Type": "Magnetic"
        }
      ]
    }
  },
  "Solver": {
    "Order": 2,
    "Device": "CPU",
    "M

In [9]:
my_config.save_config("tunable_xmon.json")